In [1]:
from pydicom import dcmread
import numpy as np
import os
import matplotlib.pyplot as plt
import re
import nibabel as nib
import SimpleITK as sitk
from helpers import *

In [6]:
class filtering:
    def __init__(self, segmentation): 
        self.segmentation=segmentation
        
    def get(self):
        array_1d = self.segmentation.flatten()
        unique_values, counts = np.unique(array_1d, return_counts=True)
        label_arrays = []
        for value in unique_values: 
            label_array = np.copy(self.segmentation) 
            # label_array[np.where(label_array != value)] = 0 
            label_array = np.where(label_array==value, label_array, 0)
            
            label_arrays.append(label_array)
        return label_arrays
            
seg_raw = sitk.ReadImage("D:/Documents/GitHub/VascuIAR/DeepLearning/data/MM_WHS/seg_res/1006/Segmentation.nii", sitk.sitkFloat32)
seg = sitk.GetArrayFromImage(seg_raw)
filtering_instance = filtering(seg)
label_arrays = filtering_instance.get()
explore_3D_array(label_arrays[2])
label_arrays[2].shape

interactive(children=(IntSlider(value=123, description='SLICE', max=247), Output()), _dom_classes=('widget-int…

(248, 512, 512)

In [4]:
class_data = {
    'class1':{
        'visible': False,
                'color': '#FFFFFF',
            },
            'class2':{
                'visible': True,
                'color': '#FFFFFF',
            },
            'class3':{
                'visible': False,
                'color': '#FFFFFF',
            },
            'class4':{
                'visible': False,
                'color': '#FFFFFF',
            },
            'class5':{
                'visible': False,
                'color': '#FFFFFF',
            },
            'class6':{
                'visible': False,
                'color': '#FFFFFF',
            },
            'class7':{
                'visible': False,
                'color': '#FFFFFF',
            },
            'class8':{
                'visible': False,
                'color': '#FFFFFF',
            },
            'class9':{
                'visible': False,
                'color': '#FFFFFF',
            },
            'class10':{
                'visible': False,
                'color': '#FFFFFF',
            },
            'class11':{
                'visible': False,
                'color': '#FFFFFF',
            }
        }



In [10]:
stacked_arrays = []
for index, (class_name, data) in enumerate(class_data.items()):
    if data['visible'] == True:
        seg = label_arrays[index][:, :, 150]
        stacked_arrays.append(seg)

combined_seg = np.vstack(stacked_arrays)
plt.imsave("temp2.png", combined_seg, cmap='gray')